In [1]:
import pandas as pd
import sqlite3 as sq
from datetime import datetime

In [2]:
rs_stocks = pd.read_csv('../src/rs_stocks.csv')
rs_stocks

,Rank,Ticker,Sector,Industry,Exchange,Relative Strength,Percentile,1 Month Ago,3 Months Ago,6 Months Ago
0,1,HLGN,Utilities,Utilities—Renewable,NYSE,564.04,99,99,5,1
1,2,AHG,Financial Services,Credit Services,NASDAQ,444.95,99,99,15,8
2,3,TPST,Healthcare,Biotechnology,NASDAQ,434.67,99,3,4,46
3,4,CABA,Healthcare,Biotechnology,NASDAQ,420.83,99,99,99,99
4,5,SYBX,Healthcare,Biotechnology,NASDAQ,334.62,99,10,6,9
...,...,...,...,...,...,...,...,...,...,...
6490,6491,SFT,Consumer Cyclical,Auto & Truck Dealerships,NASDAQ,4.42,0,7,34,4
6491,6492,CRKN,Basic Materials,Specialty Chemicals,NASDAQ,4.20,0,0,4,1
6492,6493,FFIE,Consumer Cyclical,Auto Manufacturers,NASDAQ,3.77,0,0,13,1
6493,6494,ASTI,Technology,Solar,NASDAQ,3.10,0,0,0,0


In [3]:
mkt_cap_data = pd.read_csv('../src/TV.csv')
mkt_cap_data

,Ticker,Description,Market Capitalization,Upcoming Earnings Date
0,AGOAF,ABENGOA SA,NaN,2023-09-25
1,INTG,The Intergroup Corporation,6.332624e+07,2023-09-25
2,ESMC,Escalon Medical Corp.,2.743672e+06,2023-09-25
3,ABGOF,ABENGOA SA,NaN,2023-09-25
4,RBZHF,REEBONZ HOLDING LIMITED,8.520000e+02,2023-09-25
...,...,...,...,...
18104,MAPPF,PROSTAR HLDGS INC.,2.912031e+07,NaN
18105,IHHFF,iShares II plc.,NaN,NaN
18106,CXE,MFS High Income Municipal Trust,9.678420e+07,NaN
18107,KBWY,Invesco KBW Premium Yield Equity REIT ETF,NaN,NaN


In [4]:
# Load the 'price_action' table from the 'market_data.db' database into a DataFrame named 'price_action'.
conn = sq.connect('../market_data.db')
price_action = pd.read_sql_query("SELECT * FROM price_action", conn)
conn.close()
price_action

,date,ticker,open,high,low,close,adjclose,volume
0,2020-01-02,AAOI,12.130,12.530,11.8000,12.5000,12.5000,885000.0
1,2020-01-03,AAOI,12.250,12.325,12.0100,12.1200,12.1200,498400.0
2,2020-01-06,AAOI,12.000,12.200,11.8600,12.1500,12.1500,361600.0
3,2020-01-07,AAOI,12.200,12.465,12.0550,12.4300,12.4300,330500.0
4,2020-01-08,AAOI,12.410,12.820,12.3700,12.7000,12.7000,520600.0
...,...,...,...,...,...,...,...,...
5596248,2023-10-16,GMBL,0.080,0.080,0.0663,0.0715,0.0715,12469309.0
5596249,2023-10-16,CRKN,0.335,0.375,0.3202,0.3370,0.3370,933244.0
5596250,2023-10-16,FFIE,1.110,1.160,1.0600,1.0800,1.0800,4409448.0
5596251,2023-10-16,ASTI,1.260,1.290,1.2000,1.2000,1.2000,102666.0


In [5]:
# Define the Minervini criteria with the new eliminatory criteria
def minervini_criteria_full(row):
    criteria = [
        row['Price'] > 5,
        row['Price'] < 300,
        row['Price'] > row['SMA50'],
        row['SMA50'] > row['SMA150'],
        row['SMA150'] > row['SMA200'],
        row['Price'] >= row['52Week-Low'] * 1.25,
        row['Price'] >= row['52Week-High'] * 0.85,
        row['Percentile'] >= 70,
        row['SMA200_1M_Trend'] == 1,
        row['Avg_Vol_90d'] > 100000,
        row['Avg_Vol_10d'] <= row['Avg_Vol_30d']

    ]
    return 'yes' if all(criteria) else 'no'

In [6]:
# Define the Minervini criteria with the new eliminatory criteria
def minervini_criteria_basic(row):
    criteria = [
        row['Price'] > 5,
        row['Price'] < 300,
        row['Price'] > row['SMA50'],
        row['SMA50'] > row['SMA150'],
        row['SMA150'] > row['SMA200'],
        row['Percentile'] >= 70,
        row['Avg_Vol_90d'] > 100000,

    ]
    return 'yes' if all(criteria) else 'no'

In [7]:
# Define the PowerTrend criteria
def power_trend(row, prev_power_trend):
    # Entry criteria
    if (
        row['lows_above_EMA21'] >= 10 and
        row['EMA21_above_SMA50'] >= 5 and
        row['SMA50_uptrend'] == 1 and
        row['close_above_open'] == 1
    ):
        return 'in'
    
    # Exit criteria
    if prev_power_trend == 'in' and (
        row['EMA21'] < row['SMA50'] or
        (row['close'] < row['high_last_week'] * 0.9 and row['close'] < row['SMA50'])
    ):
        return 'out'
    
    return prev_power_trend

In [8]:
screen_df =  pd.merge(price_action, mkt_cap_data, left_on='ticker', right_on='Ticker', how='left')
screen_df = pd.merge(screen_df, rs_stocks, left_on='Ticker', right_on='Ticker', how='left')
screen_df

,date,ticker,open,high,low,close,adjclose,volume,Ticker,Description,...,Upcoming Earnings Date,Rank,Sector,Industry,Exchange,Relative Strength,Percentile,1 Month Ago,3 Months Ago,6 Months Ago
0,2020-01-02,AAOI,12.130,12.530,11.8000,12.5000,12.5000,885000.0,AAOI,"Applied Optoelectronics, Inc.",...,2023-11-08,28.0,Technology,Semiconductors,NASDAQ,209.56,99.0,99.0,99.0,41.0
1,2020-01-03,AAOI,12.250,12.325,12.0100,12.1200,12.1200,498400.0,AAOI,"Applied Optoelectronics, Inc.",...,2023-11-08,28.0,Technology,Semiconductors,NASDAQ,209.56,99.0,99.0,99.0,41.0
2,2020-01-06,AAOI,12.000,12.200,11.8600,12.1500,12.1500,361600.0,AAOI,"Applied Optoelectronics, Inc.",...,2023-11-08,28.0,Technology,Semiconductors,NASDAQ,209.56,99.0,99.0,99.0,41.0
3,2020-01-07,AAOI,12.200,12.465,12.0550,12.4300,12.4300,330500.0,AAOI,"Applied Optoelectronics, Inc.",...,2023-11-08,28.0,Technology,Semiconductors,NASDAQ,209.56,99.0,99.0,99.0,41.0
4,2020-01-08,AAOI,12.410,12.820,12.3700,12.7000,12.7000,520600.0,AAOI,"Applied Optoelectronics, Inc.",...,2023-11-08,28.0,Technology,Semiconductors,NASDAQ,209.56,99.0,99.0,99.0,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5596248,2023-10-16,GMBL,0.080,0.080,0.0663,0.0715,0.0715,12469309.0,GMBL,Esports Entertainment Group Inc.,...,2023-10-12,6490.0,Consumer Cyclical,Gambling,NASDAQ,5.03,0.0,0.0,0.0,0.0
5596249,2023-10-16,CRKN,0.335,0.375,0.3202,0.3370,0.3370,933244.0,CRKN,Crown Electrokinetics Corp.,...,2023-11-10,6492.0,Basic Materials,Specialty Chemicals,NASDAQ,4.20,0.0,0.0,4.0,1.0
5596250,2023-10-16,FFIE,1.110,1.160,1.0600,1.0800,1.0800,4409448.0,FFIE,Faraday Future Intelligent Electric Inc.,...,2023-11-20,6493.0,Consumer Cyclical,Auto Manufacturers,NASDAQ,3.77,0.0,0.0,13.0,1.0
5596251,2023-10-16,ASTI,1.260,1.290,1.2000,1.2000,1.2000,102666.0,ASTI,"Ascent Solar Technologies, Inc",...,2023-11-14,6494.0,Technology,Solar,NASDAQ,3.10,0.0,0.0,0.0,0.0


In [9]:
# Add the 'Price' column representing the latest close for each stock
screen_df['Price'] = screen_df.groupby('Ticker')['close'].transform('last')

# Calculate the 50-day Simple Moving Average (SMA50) for each stock.
screen_df['SMA50'] = screen_df.groupby('Ticker')['close'].transform(lambda x: x.rolling(window=50).mean())

# Calculate the 21-day Exponential Moving Average (EMA21) for each stock.
screen_df['EMA21'] = screen_df.groupby('Ticker')['close'].transform(lambda x: x.ewm(span=21, adjust=False).mean())

# Calculate the 150-day Simple Moving Average (SMA150) for each stock.
screen_df['SMA150'] = screen_df.groupby('Ticker')['close'].transform(lambda x: x.rolling(window=150).mean())

# Calculate the 200-day Simple Moving Average (SMA200) for each stock.
screen_df['SMA200'] = screen_df.groupby('Ticker')['close'].transform(lambda x: x.rolling(window=200).mean())

# Calculate the 90-day Average Volume for each stock.
screen_df['Avg_Vol_90d'] = screen_df.groupby('Ticker')['volume'].transform(lambda x: x.rolling(window=90).mean())

# Calculate the 30-day Average Volume for each stock.
screen_df['Avg_Vol_30d'] = screen_df.groupby('Ticker')['volume'].transform(lambda x: x.rolling(window=30).mean())

# Calculate the 10-day Average Volume for each stock.
screen_df['Avg_Vol_10d'] = screen_df.groupby('Ticker')['volume'].transform(lambda x: x.rolling(window=10).mean())

# Calculate the 52-week high (highest closing price in the last 252 trading days) for each stock.
screen_df['52Week-High'] = screen_df.groupby('Ticker')['close'].transform(lambda x: x.rolling(window=252).max())

# Calculate the 52-week low (lowest closing price in the last 252 trading days) for each stock.
screen_df['52Week-Low'] = screen_df.groupby('Ticker')['close'].transform(lambda x: x.rolling(window=252).min())

# Calculate the price performance over the last 252 trading days
screen_df['12M Return'] = screen_df.groupby('Ticker')['close'].pct_change(periods=252)

# Calculate the highest high and lowest low over the last 5 daysb g
screen_df['10D_high'] = screen_df.groupby('Ticker')['high'].transform(lambda x: x.rolling(window=10).max())
screen_df['10D_low'] = screen_df.groupby('Ticker')['low'].transform(lambda x: x.rolling(window=10).min())
# Calculate the range as a percentage of the highest highb g
screen_df['10D_range_pct'] = (screen_df['10D_high'] - screen_df['10D_low']) / screen_df['10D_high'] * 100

# Identify low-risk entries
screen_df['Low-Risk_Entry'] = (screen_df['10D_range_pct'] < 8).astype(int)

# Calculate the SMA200 trend over the last month
screen_df['SMA200_1M_Trend'] = screen_df.groupby('Ticker')['SMA200'].transform(lambda x: x.diff(periods=21)).gt(0).astype(int)

# Calculate criteria for PowerTrend
screen_df['lows_above_EMA21'] = screen_df.groupby('Ticker').apply(lambda x: (x['low'] > x['EMA21']).rolling(window=10).sum()).reset_index(level=0, drop=True)
screen_df['EMA21_above_SMA50'] = screen_df.groupby('Ticker').apply(lambda x: (x['EMA21'] > x['SMA50']).rolling(window=5).sum()).reset_index(level=0, drop=True)
screen_df['SMA50_uptrend'] = screen_df.groupby('Ticker')['SMA50'].transform(lambda x: x.diff().rolling(window=21).sum()).gt(0).astype(int)
screen_df['close_above_open'] = (screen_df['close'] > screen_df['open']).astype(int)
screen_df['high_last_week'] = screen_df.groupby('Ticker')['high'].transform(lambda x: x.rolling(window=5).max())

# Apply the Minervini criteria to each row
screen_df['Minervini_basic'] = screen_df.apply(minervini_criteria_basic, axis=1)
screen_df['Minervini_full'] = screen_df.apply(minervini_criteria_full, axis=1)

# Apply the PowerTrend criteria to each row
prev_power_trend = 'out'
power_trends = []

for _, row in screen_df.iterrows():
    current_power_trend = power_trend(row, prev_power_trend)
    power_trends.append(current_power_trend)
    prev_power_trend = current_power_trend
    
# Apply the Minervini criteria to each row
screen_df['PowerTrend'] = power_trends

# Create the 'screen_df' DataFrame with the calculated metrics.
metrics = ['Ticker', 'EMA21', 'SMA50', 'SMA150', 'SMA200', 'Avg_Vol_10d', 'Avg_Vol_30d', 'Avg_Vol_90d', '52Week-High', 
           '52Week-Low', 'Minervini_basic', 'Minervini_full', 'PowerTrend', 'Low-Risk_Entry', 'Market Capitalization',
           'Upcoming Earnings Date', 'Industry', 'Percentile', '1 Month Ago', '3 Months Ago', '6 Months Ago']
screen_df = screen_df.drop_duplicates(subset='Ticker', keep='last')[metrics]
screen_df

,Ticker,EMA21,SMA50,SMA150,SMA200,Avg_Vol_10d,Avg_Vol_30d,Avg_Vol_90d,52Week-High,52Week-Low,...,Minervini_full,PowerTrend,Low-Risk_Entry,Market Capitalization,Upcoming Earnings Date,Industry,Percentile,1 Month Ago,3 Months Ago,6 Months Ago
55375,NSTC,10.368905,10.363360,10.226553,10.175265,25030.0,8.436667e+03,6.910000e+03,11.550000,9.8100,...,no,out,1,1.512016e+08,NaN,NaN,NaN,NaN,NaN,NaN
104975,SBIG,0.249772,0.335100,0.549633,0.593415,149720.0,1.681933e+05,2.090022e+05,1.740000,0.2000,...,no,out,0,5.451444e+06,2023-11-13,NaN,NaN,NaN,NaN,NaN
5241302,NSTD,10.479384,10.438200,10.267747,10.215835,16976.7,3.095890e+04,1.431186e+04,10.600000,9.8100,...,no,out,1,1.478143e+08,NaN,NaN,NaN,NaN,NaN,NaN
5241327,BKSC,12.129024,13.244800,14.303467,14.839700,67407.7,2.970923e+04,1.245863e+04,17.150000,10.4000,...,no,out,0,5.946855e+07,2023-10-12,NaN,NaN,NaN,NaN,NaN
5241341,KRBP,1.477259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,no,out,0,2.794500e+05,2023-11-13,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5596248,GMBL,0.100554,0.132430,1.146150,2.681913,11555740.9,1.087458e+07,7.727989e+06,15.800000,0.0715,...,no,out,0,5.681818e+06,2023-10-12,Gambling,0.0,0.0,0.0,0.0
5596249,CRKN,0.573514,1.177260,5.656020,8.165715,1576844.4,2.222998e+06,1.657061e+06,20.580000,0.3370,...,no,out,0,2.799249e+06,2023-11-10,Specialty Chemicals,0.0,0.0,4.0,1.0
5596250,FFIE,2.506649,8.144500,17.508433,25.244125,4224214.8,6.811762e+06,3.019098e+06,86.400002,1.0500,...,no,out,0,2.398145e+07,2023-11-20,Auto Manufacturers,0.0,0.0,13.0,1.0
5596251,ASTI,3.494669,8.265500,30.974433,72.906025,264726.6,3.964921e+05,1.492837e+05,1024.000000,1.1800,...,no,out,0,5.770567e+06,2023-11-14,Solar,0.0,0.0,0.0,0.0


In [10]:
today = datetime.now().strftime('%Y-%m-%d')
nome_arquivo = f'../output/screen_{today}.xlsx'
screen_df.to_excel(nome_arquivo, engine='xlsxwriter')

In [ ]:
# Get the current date
current_date = datetime.now().strftime('%d-%m-%Y')
table_name = f"screen-{current_date}"

# Connect to the SQLite database
conn = sq.connect('../market_data.db')

# Save the screen_df to the database with the table name based on the current date
screen_df.to_sql(table_name, conn, if_exists='replace', index=False)

# Close the database connection
conn.close()